In [ ]:
# !pip3 install -U google-cloud-aiplatform
# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

# Inner training script

In [ ]:
%%writefile training_script.py

# Source: https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom

import argparse
import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds

import hypertune

IMG_WIDTH = 128

def normalize_img(image):
    """Normalizes image.

    * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
    * Casts values from `uint8` to `float32`
    * Scales values from [0, 255] to [0, 1]

    Returns:
      A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
    """
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    return image / 255.


def normalize_img_and_label(image, label):
    """Normalizes image and label.

    * Performs normalize_img on image
    * Passes through label unchanged

    Returns:
      Tuple (image, label) where
      * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
        channels)
      * label is an unchanged integer [0, 4] representing flower type
    """
    return normalize_img(image), label

def get_args():
  """Argument parser.
  Returns:
    Dictionary of arguments.
  """
  parser = argparse.ArgumentParser(description='Flower classification sample')
  parser.add_argument(
      '--tfds',
      default=None,
      help='The tfds URI from https://www.tensorflow.org/datasets/ to load the data from')

  parser.add_argument('--lr', type=float, default=0.01)
  parser.add_argument('--units', type=int, default=4)
  parser.add_argument('--activation', type=str, default='relu')
  parser.add_argument('--batch_size', type=int, default=128)

  args = parser.parse_args()
  return args

class HPTCallback(tf.keras.callbacks.Callback):
    
    def __init__(self):
        super().__init__()
        self._hp_tune_reporter = hypertune.HyperTune()

    def on_epoch_end(self, epoch, logs=None):
        self._hp_tune_reporter.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='loss',
            metric_value=logs['loss'],
            global_step=epoch)

# Training settings
args = get_args()

if 'AIP_MODEL_DIR' not in os.environ:
    raise KeyError(
        'The `AIP_MODEL_DIR` environment variable has not been' +
        'set. See https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom/training'
    )
output_directory = os.environ['AIP_MODEL_DIR']

logging.info('Loading and preprocessing data ...')
dataset = tfds.load(args.tfds,
                    split='train',
                    try_gcs=True,
                    shuffle_files=True,
                    as_supervised=True)
dataset = dataset.map(normalize_img_and_label,
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(args.batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

logging.info('Creating and training model ...')
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(args.units,
                           3,
                           padding='same',
                           activation='relu',
                           input_shape=(IMG_WIDTH, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(args.units * 2, 3, padding='same', activation=args.activation),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(args.units * 3, 3, padding='same', activation=args.activation),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=args.activation),
    tf.keras.layers.Dense(5)  # 5 classes
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=args.lr),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
model.fit(dataset, epochs=10, callbacks=[HPTCallback()])

logging.info(f'Exporting SavedModel to: {output_directory}')
# Add softmax layer for intepretability
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
probability_model.save(output_directory)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

# Construct the Custom Training Job To pass into HP Tuning

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

aiplatform.init(project='sashaproject-1',
                staging_bucket='gs://ucaip-mb-sasha-dev')

job = aiplatform.CustomJob.from_local_script(
    display_name='my-job',
    script_path='training_script.py',
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest",
    requirements=["gcsfs==0.7.1", "cloudml-hypertune"],
    args=["--tfds", "tf_flowers:3.*.*"],
    replica_count=1,
    accelerator_count=1,
    accelerator_type='NVIDIA_TESLA_K80',
)

# Run HP Tuning Jub with constructed Custom Job

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name='hp-test',
    custom_job=job,
    metric_spec={'loss': 'minimize'},
    parameter_spec={
        'lr': hpt.DoubleParameterSpec(min=0.001, max=0.1, scale='log'),
        'units': hpt.IntegerParameterSpec(min=4, max=32, scale='linear'),
        'activation': hpt.CategoricalParameterSpec(values=['relu', 'sigmoid', 'elu', 'selu', 'tanh']),
        'batch_size': hpt.DiscreteParameterSpec(values=[32, 64, 128], scale='linear')
    },
    max_trial_count=128,
    parallel_trial_count=8,    
    )

hp_job.run()
 

# You can run the Custom Job stand alone as well

In [ ]:
job.run()

Alternatively, you can create a Custom Job from worker pools specs. We'll reuse the Python Package we created fromt the local script above. 

In [ ]:
worker_pool_specs = [
   {
       "replica_count": 1,
       "machine_spec": {
         "machine_type": 'n1-standard-4',
       },
       "python_package_spec": {
           "executor_image_uri": 'us-docker.pkg.dev/cloud-aiplatform/training/tf-cpu.2-4:latest',
           "package_uris": job.job_spec.worker_pool_specs[0].python_package_spec.package_uris,
           "python_module": "aiplatform_custom_trainer_script.task",
           "args": ["--tfds", "tf_flowers:3.*.*"]
       },
   }
]

In [ ]:
job = aiplatform.CustomJob(display_name='test-from_worker_pool_spec',
                           worker_pool_specs=worker_pool_specs)
job.run()
